In [2]:
import io
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats
import scipy.special
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook')
from sklearn.model_selection import train_test_split
import os

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import balanced_accuracy_score

from sklearn import metrics
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import TomekLinks


from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE

##### from collections import Counter
from sklearn.datasets import make_classification
from sklearn.cluster import MiniBatchKMeans
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTENC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

from imblearn.ensemble import EasyEnsembleClassifier

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


import scipy.stats as st
import math


#Import library
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import BaggingClassifier
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.preprocessing import LabelEncoder


import numpy as np
from adacost import AdaCostClassifier 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import pandas as pd
seed=45


## Training Set

In [3]:
data=pd.read_csv('/Users/ibrain/Downloads/Cognitive Neuroscience Work/Cognition and Lifestyle R Coding/Cognition_Lifestyle/Imbalanced Data/training_set.csv')
data=data.iloc[:,1:]
data.head()

X = data.drop(columns='CIMEMLOS')
Y = data['CIMEMLOS']


## Computing Information Ratio

In [4]:
ratio = sum(Y==0)/sum(Y==1)
print('Train_Pos:Train_Neg = {:}:{:} = {:.2f}'.format(sum(Y==0), sum(Y==1), ratio))

Train_Pos:Train_Neg = 15004:1805 = 8.31


## Cross Validation and Testing on Validation Dataset

In [9]:
clf_ada = AdaCostClassifier(FNcost=2,
                            FPcost=0.12,
                            random_state=2018)

## Final Training

In [10]:
clf_ada.fit(X, Y)


AdaCostClassifier(FNcost=2, FPcost=0.12, random_state=2018)

## Evaluation Set

In [11]:
eval_set=pd.read_csv('/Users/ibrain/Downloads/Cognitive Neuroscience Work/Cognition and Lifestyle R Coding/Cognition_Lifestyle/Imbalanced Data/evaluation_set.csv')
eval_set=eval_set.iloc[:,1:]
eval_set.head()
print(eval_set.shape)

eval_X = eval_set.drop(columns='CIMEMLOS')
eval_Y = eval_set['CIMEMLOS']

(2401, 18)


In [12]:
y_pred=clf_ada.predict(eval_X)
print(confusion_matrix(eval_Y, y_pred))
print(classification_report(eval_Y, y_pred))
y_pred_proba=clf_ada.predict_proba(eval_X)[::,1]

# Calculation of AUC
auc=metrics.roc_auc_score(eval_Y, y_pred_proba)
print(auc)

[[1805  315]
 [ 158  123]]
              precision    recall  f1-score   support

           0       0.92      0.85      0.88      2120
           1       0.28      0.44      0.34       281

    accuracy                           0.80      2401
   macro avg       0.60      0.64      0.61      2401
weighted avg       0.84      0.80      0.82      2401

0.7085619082790574


## Complete Training set by merging Training and Validation Set

In [13]:
train_eval=[data, eval_set]
complete_train=pd.concat(train_eval)
print(len(complete_train))

19210


In [14]:
complete_train_X = complete_train.drop(columns='CIMEMLOS')
complete_train_Y = complete_train['CIMEMLOS']

In [15]:
complete_train_X.head()

,SEXVAR,X.AGEG5YR,X.BMI5,EXERANY2,SLEPTIM1,ADDEPEV3,DIABETE4,CVDSTRK3,smok_everyday,smok_somday,smok_no,SES,White,Black,Hispanic,Multi,Other
0,0,10,28.29,1,7,0,1,0,0,0,1,11,1,0,0,0,0
1,1,11,37.97,0,6,0,1,0,0,0,1,5,1,0,0,0,0
2,0,9,19.84,1,8,0,0,0,1,0,0,14,1,0,0,0,0
3,0,7,21.11,0,4,1,1,0,0,0,1,6,1,0,0,0,0
4,1,11,20.53,0,4,0,0,0,1,0,0,12,1,0,0,0,0


In [16]:
complete_train_Y.head()

0    0
1    0
2    0
3    0
4    0
Name: CIMEMLOS, dtype: int64

In [17]:
clf_ada.fit(complete_train_X, complete_train_Y)

AdaCostClassifier(FNcost=2, FPcost=0.12, random_state=2018)

## Final Testing 

In [18]:
test_data=pd.read_csv('/Users/ibrain/Downloads/Cognitive Neuroscience Work/Cognition and Lifestyle R Coding/Cognition_Lifestyle/Imbalanced Data/testing_set.csv')
test_data=test_data.iloc[:,1:]
test_data.head()

test_X = test_data.drop(columns='CIMEMLOS')
test_Y = test_data['CIMEMLOS']

print(test_X.shape, test_Y.shape)


(4802, 17) (4802,)


## Final Testing Metrics 

In [19]:
y_pred=clf_ada.predict(test_X)

print(confusion_matrix(test_Y, y_pred))
print(classification_report(test_Y, y_pred))
y_pred_proba=clf_ada.predict_proba(test_X)[::,1]

    # Calculation of AUC
auc=metrics.roc_auc_score(test_Y, y_pred_proba)
print(auc)

[[3631  666]
 [ 287  218]]
              precision    recall  f1-score   support

           0       0.93      0.85      0.88      4297
           1       0.25      0.43      0.31       505

    accuracy                           0.80      4802
   macro avg       0.59      0.64      0.60      4802
weighted avg       0.86      0.80      0.82      4802

0.7135848865314737
